In [1]:
import numpy as np
import json, requests
import pandas as pd
import random
import cmlapi
import os

### Overview

the notebook will:
1. take input format for python endpoint
2. extract model url and model access key
3. reading input data, will take a single sample of link/location combination 
4. then take a random sample of 25 contiguous rows
5. Convert this into json format through 'dataframe_to_json' function
6. Put all pieces together for json payload required by endpoint
7. Send prediction request
8. 'Unpack' response

In [2]:
window = 24

In [3]:
def dataframe_to_json(df):
    """Create a dictionary that will later be converted to a JSON object
    ensure that 
    """
    data = {
        'rx_gbs': df['rx_gbs'].iloc[-(window+1):].tolist(),
        'tx_gbs': df['tx_gbs'].iloc[-(window+1):].tolist(),
        'time': [df['time'].iloc[-1]],  # Only the last time entry
        'link': [df['link'].iloc[-1]],  # Only the last link entry
        'location': [df['location'].iloc[-1]]  # Only the last location entry
    }
    
    # build embedded dictionary step 1
    request_dict = {"request":data}

    # access key will be end point specific
    BackDict = {"accessKey":model_key}
    BackDict.update(request_dict)
    request_dict=BackDict
    
    return request_dict

In [4]:
df = pd.read_csv('data/netstats_hourly_4_3.csv')
#df.head()

In [8]:
model_name = input("Enter the name of the model: ")

Enter the name of the model:  lstm-test2


In [9]:
## Need to get the below prgramatically in the future iterations
client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
target_model = client.list_all_models(search_filter=json.dumps({"name": model_name}))
model_key = target_model.models[0].access_key
model_url = os.getenv("CDSW_API_URL").replace("https://", "https://modelservice.").replace("/api/v1", "/model")

In [10]:
target_model.models[0].crn

'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:d3dd4a0b-b3ab-4a08-98ab-7704bb9076ed/69828e40-011c-4d6a-9d74-fc65d58e7010'

Pick a random link / location

In [11]:
sample_df = df[['link','location']].drop_duplicates().sample()
link_pick = sample_df['link'].values[0]
location_pick = sample_df['location'].values[0]
print(sample_df)

      link   location
10940  ER2  Frankfurt


In [12]:
# test_df has only one combintation  of link_pick and location_pick
test_df = df[(df.link == link_pick) & (df.location == location_pick)]

In [13]:
# pick a random segment of size window + 1
start_row = np.random.randint(0, len(test_df) - (window+1))
random_segment = df.iloc[start_row:start_row + (window+1)]

In [14]:
# confirm size
random_segment.shape

(25, 6)

In [15]:
random_segment.head()

,link,location,link_loc,time,rx_gbs,tx_gbs
503,CenturyLink,US-DA3,CenturyLink_US-DA3,2024-04-01 12:00:00,5.839567,1.88097
504,CenturyLink,US-DA3,CenturyLink_US-DA3,2024-04-01 13:00:00,2.789959,0.78486
505,CenturyLink,US-DA3,CenturyLink_US-DA3,2024-04-01 14:00:00,0.000000,0.00000
506,CenturyLink,US-DA3,CenturyLink_US-DA3,2024-04-01 15:00:00,0.000000,0.00000
507,CenturyLink,US-DA3,CenturyLink_US-DA3,2024-04-01 16:00:00,0.000000,0.00000


In [16]:
# convert input to json
request_dict = dataframe_to_json(random_segment)

In [17]:
#test_inputs

Step below puts it into required payload for endpoint

In [18]:
# # build embedded dictionary step 1
# request_dict = {"request":test_inputs}

# # access key will be end point specific
# BackDict = {"accessKey":model_key}
# BackDict.update(request_dict)
# request_dict=BackDict

In [19]:
r = requests.post(model_url, data=json.dumps(request_dict), headers={'Content-Type': 'application/json'})

In [20]:
r.json()

{'success': True,
 'response': {'model_deployment_crn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:d3dd4a0b-b3ab-4a08-98ab-7704bb9076ed/83221989-428d-43d2-9611-607d822c9bdd',
  'prediction': {'rx_bytes': [19.57953643798828,
    21.920862197875977,
    22.30994415283203,
    22.806299209594727,
    23.943214416503906,
    23.513851165771484],
   'time': ['2024-04-02T13:00:00',
    '2024-04-02T14:00:00',
    '2024-04-02T15:00:00',
    '2024-04-02T16:00:00',
    '2024-04-02T17:00:00',
    '2024-04-02T18:00:00'],
   'tx_bytes': [6.19659948348999,
    3.352660655975342,
    1.1895450353622437,
    1.2256425619125366,
    1.360798954963684,
    -1.3282729387283325]},
  'uuid': '88e57f2c-8384-4ed8-8433-e04e6a099121'},
 'ReplicaID': 'lstm-test2-5-16-6ff6b87b44-l46l2',
 'Size': 647,
 'StatusCode': 200}

Get the model response

In [26]:
r.json()['response']['prediction']['time']

['2024-04-02T13:00:00',
 '2024-04-02T14:00:00',
 '2024-04-02T15:00:00',
 '2024-04-02T16:00:00',
 '2024-04-02T17:00:00',
 '2024-04-02T18:00:00']

In [22]:
r.json()['response']['prediction']['rx_bytes']

[19.57953643798828,
 21.920862197875977,
 22.30994415283203,
 22.806299209594727,
 23.943214416503906,
 23.513851165771484]

In [23]:
r.json()['response']['prediction']['tx_bytes']

[6.19659948348999,
 3.352660655975342,
 1.1895450353622437,
 1.2256425619125366,
 1.360798954963684,
 -1.3282729387283325]

In [29]:
r.json()['response']['uuid']

'88e57f2c-8384-4ed8-8433-e04e6a099121'

In [20]:
type(rx_response)

list

In [23]:
type(rx_response[0])

list